In [7]:
import pandas as pd
import sqlalchemy
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
import uuid

In [2]:
conn = sqlalchemy.create_engine(os.getenv('BILL_DB_CONN_STRING')).connect()

In [3]:
# first populate bill_gpt.bill_details_v2 with the 3 known bills
bill_details_v1 = pd.read_sql('select * from bill_gpt.bill_details', conn)

In [5]:
bill_details_v1.rename(columns={'bill_id': 'bill_alias'}, inplace=True)

In [8]:
bill_details_v1['bill_id'] = bill_details_v1['bill_alias'].apply(lambda x: uuid.uuid4())

In [9]:
bill_details_v1

,bill_alias,bill_title,summary,wiki_link,full_text_link,bill_id
0,fra,Fiscal Responsibility Act of 2023,The Fiscal Responsibility Act of 2023 was pass...,https://en.wikipedia.org/wiki/Fiscal_Responsib...,https://www.congress.gov/118/bills/hr3746/BILL...,449f4266-6333-480c-9b22-57b8cc88b053
1,ira,Fiscal Responsibility Act of 2023,The Inflation Reduction Act of 2022 was passed...,https://en.wikipedia.org/wiki/Inflation_Reduct...,https://www.congress.gov/117/bills/hr5376/BILL...,a70197aa-7169-4e1a-8e8a-ea34db99ed5f
2,chips,CHIPS and Science Act,The CHIPS and Science Act was passed in August...,https://en.wikipedia.org/wiki/CHIPS_and_Scienc...,https://www.congress.gov/117/bills/hr4346/BILL...,58a9aafb-1d53-4e43-9cfa-086fdc620aea


In [15]:
conn.commit()

In [14]:
for i, row in bill_details_v1.iterrows():
    stmt = sqlalchemy.text(f"insert into bill_gpt.bill_details_v2(bill_id, bill_title, bill_alias, summary, wiki_link, full_text_link) "
                 f"values (:bill_id, :bill_title, :bill_alias, :summary, :wiki_link, :full_text_link)")
    stmt = stmt.bindparams(bill_id=row['bill_id'], bill_title=row['bill_title'], bill_alias=row['bill_alias'], summary=row['summary'], wiki_link=row['wiki_link'], full_text_link=row['full_text_link'])
    conn.execute(stmt)

In [29]:
conn.commit()

In [16]:
chips_q_a = pd.read_sql('select * from bill_gpt.chips_questions_and_answers', conn)

In [25]:
chips_q_a

,id,created_at,question,answer,bill_id,question_id
0,2,2023-09-24 17:24:21.226831+00:00,What are the provisions related to semiconduct...,The provisions related to semiconductors in th...,58a9aafb-1d53-4e43-9cfa-086fdc620aea,e03f6bce-7362-4267-a1d9-e0768edee092
1,3,2023-09-24 17:24:41.122989+00:00,What programs and initiatives are included in ...,The bill includes several programs and initiat...,58a9aafb-1d53-4e43-9cfa-086fdc620aea,9285dd07-35f9-4d03-b63d-5dad4632b747


In [22]:
chips_q_a['bill_id'] = bill_details_v1[bill_details_v1['bill_alias'] =='chips'].iloc[0]['bill_id']

In [24]:
chips_q_a['question_id'] = chips_q_a['question'].apply(lambda x: uuid.uuid4())

In [30]:
for i, row in chips_q_a.iterrows():
    stmt = sqlalchemy.text(f"insert into bill_gpt.questions_asked(question_id, bill_id, question, answer) "
                 f"values (:question_id, :bill_id, :question, :answer)")
    stmt = stmt.bindparams(question_id=row['question_id'], bill_id=row['bill_id'], question=row['question'], answer=row['answer'])
    conn.execute(stmt)
conn.commit()

In [33]:
fra_q_a = pd.read_sql('select * from bill_gpt.fra_questions_and_answers', conn)

In [34]:
fra_q_a

,id,created_at,question,answer
0,1,2023-09-24 15:55:46.927558+00:00,What is the purpose of the bills and legislati...,The purpose of the bills and legislative docum...
1,2,2023-09-24 15:56:06.159616+00:00,What is the purpose of the bill?,The purpose of the bill is to establish limita...
2,3,2023-09-24 15:56:50.135586+00:00,What are the specific provisions and amendment...,The specific provisions and amendments include...
3,4,2023-09-24 15:57:45.863575+00:00,How are environmental impact assessments condu...,Environmental impact assessments are conducted...


In [35]:
fra_q_a['bill_id'] = bill_details_v1[bill_details_v1['bill_alias'] =='fra'].iloc[0]['bill_id']

In [36]:
fra_q_a['question_id'] = fra_q_a['question'].apply(lambda x: uuid.uuid4())

In [37]:
for i, row in fra_q_a.iterrows():
    stmt = sqlalchemy.text(f"insert into bill_gpt.questions_asked(question_id, bill_id, question, answer) "
                 f"values (:question_id, :bill_id, :question, :answer)")
    stmt = stmt.bindparams(question_id=row['question_id'], bill_id=row['bill_id'], question=row['question'], answer=row['answer'])
    conn.execute(stmt)
conn.commit()